In [36]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('keywords.csv')
df2 = pd.read_csv('movies_metadata.csv')

/var/folders/w6/pmmfklps59z60whl_87vd6gm0000gn/T/ipykernel_63540/938889069.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('movies_metadata.csv')


In [92]:
df1['id'] = df1['id'].astype(str)
df2['id'] = df2['id'].astype(str)
df = df1.merge(df2, on='id', how='outer')
df.shape

(46486, 25)

In [94]:
#df = df.head(20000)
df= df[df['original_language'] == 'en']

In [93]:
sum(df['original_language'] == 'en')

32852

In [95]:
sum(df.duplicated(subset='original_title'))

2359

In [96]:
df = df.drop(columns=['homepage'])
df = df.drop(columns=['imdb_id'])
df = df.drop(columns=['spoken_languages'])
df = df.drop(columns=['video'])
df = df.drop(columns=['title'])
df = df.drop(columns=['belongs_to_collection'])
df = df.drop(columns=['status'])
df = df.drop(columns=['poster_path'])
df = df.drop(columns=['budget'])

df.dtypes

id                       object
keywords                 object
adult                    object
genres                   object
original_language        object
original_title           object
overview                 object
popularity               object
production_companies     object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
tagline                  object
vote_average            float64
vote_count              float64
dtype: object

In [97]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

df.dtypes

id                        int64
keywords                 object
adult                    object
genres                   object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
tagline                  object
vote_average            float64
vote_count              float64
dtype: object

In [98]:
df.isna().sum()

id                          0
keywords                    0
adult                       0
genres                      0
original_language           0
original_title              0
overview                   72
popularity                  3
production_companies        3
production_countries        3
release_date               68
revenue                     3
runtime                     9
tagline                 14683
vote_average                3
vote_count                  3
dtype: int64

In [99]:
from sklearn.impute import SimpleImputer

# Optionally, for numerical columns with NaNs:
num_imputer = SimpleImputer(strategy='mean')  # or median, depending on your preference
num_cols = df.select_dtypes(include=['float64']).columns

df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Check for remaining NaNs
print(df.isna().sum())

id                          0
keywords                    0
adult                       0
genres                      0
original_language           0
original_title              0
overview                   72
popularity                  0
production_companies        3
production_countries        3
release_date               68
revenue                     0
runtime                     0
tagline                 14683
vote_average                0
vote_count                  0
dtype: int64


In [100]:
from sklearn.preprocessing import LabelEncoder
# Create a dictionary of label encoders for each column
label_encoders = {}
categorical_columns = ['keywords', 'adult', 'genres', 'original_language', 'original_title', 'overview', 'production_companies', 'production_countries', 'release_date', 'tagline']

for col in categorical_columns:
    le = LabelEncoder()
    # Fit the encoder and transform the column (handle missing values as a category)
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

In [101]:
df.dtypes

id                        int64
keywords                  int64
adult                     int64
genres                    int64
original_language         int64
original_title            int64
overview                  int64
popularity              float64
production_companies      int64
production_countries      int64
release_date              int64
revenue                 float64
runtime                 float64
tagline                   int64
vote_average            float64
vote_count              float64
dtype: object

In [102]:
df.head()

,id,keywords,adult,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,tagline,vote_average,vote_count
0,100,13526,0,2577,0,13769,1259,4.607860,5935,837,8852,3897569.0,105.0,408,7.5,1671.0
2,10001,4784,0,2587,0,30150,8183,2.562888,15132,95,7197,0.0,91.0,3787,4.5,46.0
3,100010,9844,0,1250,0,8486,5022,0.769266,8353,1380,1482,0.0,116.0,11873,6.0,1.0
5,10002,6188,0,1561,0,15255,15354,3.095584,12782,837,6828,5794184.0,104.0,17778,6.7,62.0
6,100024,6532,0,1737,0,3614,1643,1.099911,0,1257,12711,0.0,100.0,9350,4.3,13.0


In [103]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [104]:
for col in categorical_columns:
    df[col] = label_encoders[col].inverse_transform(df[col].round().astype(int))

df.head()

,id,keywords,adult,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,tagline,vote_average,vote_count
0,100.0,"[{'id': 502, 'name': 'ambush'}, {'id': 567, 'n...",False,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",en,"Lock, Stock and Two Smoking Barrels",A card sharp and his unwillingly-enlisted frie...,4.607860,"[{'name': 'Handmade Films Ltd.', 'id': 146}, {...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1998-03-05,3897569.0,105.0,A Disgrace to Criminals Everywhere.,7.5,1671.0
1,10001.0,"[{'id': 1815, 'name': 'atomic bomb'}, {'id': 2...",False,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",en,Young Einstein,Albert Einstein is the son of a Tasmanian appl...,2.562888,"[{'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'AU', 'name': 'Australia'}]",1988-12-15,0.0,91.0,E=mc²,4.5,46.0
2,100010.0,"[{'id': 3203, 'name': 'pilot'}, {'id': 3588, '...",False,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",en,Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",0.769266,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1940-12-27,0.0,116.0,"T-H-R-I-L-L AMERICA! Here come The Flying ""HEL...",6.0,1.0
3,10002.0,"[{'id': 212, 'name': 'london england'}, {'id':...",False,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,Mona Lisa,"George has just been released from prison, and...",3.095584,"[{'name': 'The Criterion Collection', 'id': 10...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1986-06-13,5794184.0,104.0,nan,6.7,62.0
4,100024.0,"[{'id': 222018, 'name': 'dangerous side effect...",False,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",en,Bloodwork,A couple of college students decide to sign up...,1.099911,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-05-10,0.0,100.0,Not Dying Doesn't Mean You're Alive,4.3,13.0


In [105]:
df.isna().sum()

id                      0
keywords                0
adult                   0
genres                  0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
tagline                 0
vote_average            0
vote_count              0
dtype: int64

In [106]:
df.head()

,id,keywords,adult,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,tagline,vote_average,vote_count
0,100.0,"[{'id': 502, 'name': 'ambush'}, {'id': 567, 'n...",False,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",en,"Lock, Stock and Two Smoking Barrels",A card sharp and his unwillingly-enlisted frie...,4.607860,"[{'name': 'Handmade Films Ltd.', 'id': 146}, {...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1998-03-05,3897569.0,105.0,A Disgrace to Criminals Everywhere.,7.5,1671.0
1,10001.0,"[{'id': 1815, 'name': 'atomic bomb'}, {'id': 2...",False,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",en,Young Einstein,Albert Einstein is the son of a Tasmanian appl...,2.562888,"[{'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'AU', 'name': 'Australia'}]",1988-12-15,0.0,91.0,E=mc²,4.5,46.0
2,100010.0,"[{'id': 3203, 'name': 'pilot'}, {'id': 3588, '...",False,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",en,Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",0.769266,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1940-12-27,0.0,116.0,"T-H-R-I-L-L AMERICA! Here come The Flying ""HEL...",6.0,1.0
3,10002.0,"[{'id': 212, 'name': 'london england'}, {'id':...",False,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,Mona Lisa,"George has just been released from prison, and...",3.095584,"[{'name': 'The Criterion Collection', 'id': 10...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1986-06-13,5794184.0,104.0,nan,6.7,62.0
4,100024.0,"[{'id': 222018, 'name': 'dangerous side effect...",False,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",en,Bloodwork,A couple of college students decide to sign up...,1.099911,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-05-10,0.0,100.0,Not Dying Doesn't Mean You're Alive,4.3,13.0


In [107]:
df.dtypes

id                      float64
keywords                 object
adult                    object
genres                   object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
tagline                  object
vote_average            float64
vote_count              float64
dtype: object

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df['combined_features'] = df['genres'] + ' ' + df['keywords'] + ' ' + df['overview'] + ' ' + df['tagline']

tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

: 

In [90]:
def get_recommendations(movie_name, cosine_sim=cosine_sim):
    idx = df.index[df['original_title'].str.contains(movie_name, case=False, na=False)].tolist()[0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['original_title'].iloc[movie_indices]

In [91]:
movie_name = 'Young Einstein'
recommended_movies = get_recommendations(movie_name)

print("Top 5 movies similar to {}: \n".format(movie_name))
print(recommended_movies)

Top 5 movies similar to Young Einstein: 

3264                       I.Q.
7847     Einstein and Eddington
13349                  Infinity
11292     L'Histoire d'Adèle H.
19969                  D.C. Cab
Name: original_title, dtype: object
